In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

!ls /content/gdrive/My\ Drive/Dora/Bilkent/CS464/HW1 # Use YOUR OWN DIRECTORY!!


import os
import csv
import math
import random
import operator
import pdb
import time

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import scipy as scp

from timeit import default_timer as timer
from math import log, inf

np.random.seed(123)

tic = time.perf_counter()

countfiles = 0
countspam = 0

root = '/content/gdrive/My Drive/Dora/Bilkent/CS464/HW1'
csv_yTrain = os.path.join(root, 'y_train.csv')
csv_xTrain = os.path.join(root, 'x_train.csv')
csv_yTest = os.path.join(root, 'y_test.csv')
csv_xTest = os.path.join(root, 'x_test.csv')
vocab = os.path.join(root, 'vocabulary.txt')

import numpy as np
import pandas as pd

#read the csv files ot extract the feature and labels from the test and 
train_feature = pd.read_csv(csv_xTrain, header=None)
train_label = pd.read_csv(csv_yTrain, header=None)
test_feature = pd.read_csv(csv_xTest, header=None)
test_label = pd.read_csv(csv_yTest, header=None)

labels = ['0' , '1']
vocabcount = 44020 #since this quantity is given, it is directly used, however it could've also been easily calculated.

toc = time.perf_counter()
print(f"Total time is {toc - tic:0.4f} seconds")


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
vocabulary.txt	x_test.csv  x_train.csv  y_test.csv  y_train.csv
Total time is 188.1832 seconds


In [6]:
tic = time.perf_counter()

#generate the train_multinomial concatanated matrix to assign training features to their respective labels.
train_multinomial = pd.concat((train_label[0].rename('label'), train_feature), axis=1)

#count the number of spam and normal mails in the training dataset
label_counts = train_label[0].value_counts() 
normal_mails = label_counts[0]
spam_mails = label_counts[1]
total_mails = normal_mails + spam_mails

#calculate the probabilities of choosing a normal or a spam email from the dataset.
prob_normal = normal_mails/total_mails
prob_spam = spam_mails/total_mails

toc = time.perf_counter()
print(f"Total time is {toc - tic:0.4f} seconds")


Total time is 0.1258 seconds


In [7]:
tic = time.perf_counter()

#find the number of occurances of a specific word in the spam and normal datasets, and store it in occurances.
occurances = train_multinomial.groupby('label').sum()

#extract the total number of each word used in the normal and spam mails seperately.
normal_total = occurances.sum(axis=1)[0]
spam_total = occurances.sum(axis=1)[1]

#count the number of times that a particular word has occured. This will be used for the Bernoulli Model.
countoccurance = train_multinomial.mask(train_multinomial > 1, 1).groupby('label').sum()

toc = time.perf_counter()
print(f"Total time is {toc - tic:0.4f} seconds")


Total time is 12.6460 seconds


In [8]:
def confusion_results(prediction_matrix):

    #this function is defined to compute the confusion matrix values for the ML models.
    
    true_positive = 0
    true_negative = 0
    false_negative = 0
    false_positive = 0 

    for i in range(len(prediction_matrix)):
        if prediction_matrix[i] == 1 and prediction_matrix[i] == test_label[0][i] :
            true_positive += 1
        elif prediction_matrix[i] == 0 and prediction_matrix[i] == test_label[0][i] :
            true_negative += 1
        elif prediction_matrix[i] == 1 and prediction_matrix[i] != test_label[0][i] :
            false_positive += 1
        elif prediction_matrix[i] == 0 and prediction_matrix[i] != test_label[0][i] :
            false_negative += 1

    accuracy = (true_positive+true_negative)/(true_negative+true_positive+false_negative+false_positive)*100
    precision = true_positive / (true_positive + false_positive)*100

    print('Number of True Positives: ', true_positive)
    print('Number of True Negatives: ', true_negative)
    print('Number of False Positives: ', false_positive)
    print('Number of False Negatives: ', false_negative)
    print('Accuracy is ', accuracy, ' %')
    print('Precision is ', precision, ' %')

In [9]:
def MLE_predictor(email):
    
    #this function is used to calculate the posteriors from the Multinomial MLE model, given its priors.
    #the posteriror for spam and normal will be calculated and compared to reach to a prediction for a particular email document.

    yi_normaldf = (email * np.log((occurances.iloc[0])/(normal_total))).fillna(0)
    yi_spamdf = (email * np.log((occurances.iloc[1])/(spam_total))).fillna(0) 

    yi_normal = yi_normaldf.values.sum() + math.log(prob_normal)
    yi_spam = yi_spamdf.values.sum() + math.log(prob_spam)


    if yi_normal > yi_spam:
        prediction = 0
    elif yi_normal < yi_spam:
        prediction = 1
    elif yi_normal == yi_spam:
        prediction = 0

    return prediction


In [10]:
def MAP_predictor(email):
    
    #this function is used to calculate the posteriors from the MAP model, given its priors.
    #the posteriror for spam and normal will be calculated and compared to reach to a prediction for a particular email document.

    yi_normaldf = (email * np.log((occurances.iloc[0]+1)/(normal_total+44020))).fillna(0)
    yi_spamdf = (email * np.log((occurances.iloc[1]+1)/(spam_total+44020))).fillna(0) 

    yi_normal = yi_normaldf.values.sum() + math.log(prob_normal)
    yi_spam = yi_spamdf.values.sum() + math.log(prob_spam)


    if yi_normal > yi_spam:
        prediction = 0
    elif yi_normal < yi_spam:
        prediction = 1
    elif yi_normal == yi_spam:
        prediction = 0

    return prediction

In [11]:
def Bernoulli_predictor(email):

    #this function is used to calculate the posteriors from the Bernoulli model, given its priors.
    #the posteriror for spam and normal will be calculated and compared to reach to a prediction for a particular email document.

    newmail = email.mask(email > 1, 1)
    compnewmail = 1-newmail

    yi_normaldf = (newmail * np.log((countoccurance.iloc[0])/normal_mails)).fillna(0) + ((compnewmail)*(np.log(1-(countoccurance.iloc[0])/normal_mails))).fillna(0)
    yi_spamdf = (newmail * np.log((countoccurance.iloc[1])/(spam_mails))).fillna(0) + ((compnewmail)*(np.log(1-(countoccurance.iloc[1])/spam_mails))).fillna(0)

    yi_normal = yi_normaldf.values.sum() + math.log(prob_normal)
    yi_spam = yi_spamdf.values.sum() + math.log(prob_spam)

    if yi_normal > yi_spam:
        prediction = 0
    elif yi_normal < yi_spam:
        prediction = 1
    elif yi_normal == yi_spam:
        prediction = 0

    return prediction

In [13]:
prediction_matrix_MLE = []
np.warnings.filterwarnings('ignore')

tic = time.perf_counter()

#the prediction for every document will be conducted from the test features, using the corresponding model.
#predictions will be stored in its correspondig matrix, each index referring to the respective document. 

for i in range(len(test_label)):
    prediction_matrix_MLE.append(MLE_predictor(test_feature.iloc[i]))
  
confusion_results(prediction_matrix_MLE)

toc = time.perf_counter()
print(f"Total time is {toc - tic:0.4f} seconds")

Number of True Positives:  611
Number of True Negatives:  317
Number of False Positives:  8
Number of False Negatives:  150
Accuracy is  85.451197053407  %
Precision is  98.70759289176091  %
Total time is 16.1942 seconds


In [14]:
prediction_matrix_MAP = []

tic = time.perf_counter()

#the prediction for every document will be conducted from the test features, using the corresponding model.
#predictions will be stored in its correspondig matrix, each index referring to the respective document. 

for i in range(len(test_label)):
    prediction_matrix_MAP.append(MAP_predictor(test_feature.iloc[i]))
  
confusion_results(prediction_matrix_MAP)

toc = time.perf_counter()
print(f"Total time is {toc - tic:0.4f} seconds")

Number of True Positives:  751
Number of True Negatives:  315
Number of False Positives:  10
Number of False Negatives:  10
Accuracy is  98.15837937384899  %
Precision is  98.68593955321944  %
Total time is 17.4232 seconds


In [15]:
prediction_matrix_Bernoulli = []
np.warnings.filterwarnings('ignore')

tic = time.perf_counter()

#the prediction for every document will be conducted from the test features, using the corresponding model.
#predictions will be stored in its correspondig matrix, each index referring to the respective document. 

for i in range(len(test_label)):
    prediction_matrix_Bernoulli.append(Bernoulli_predictor(test_feature.iloc[i]))
  
confusion_results(prediction_matrix_Bernoulli)

toc = time.perf_counter()
print(f"Total time is {toc - tic:0.4f} seconds")

Number of True Positives:  613
Number of True Negatives:  300
Number of False Positives:  25
Number of False Negatives:  148
Accuracy is  84.06998158379373  %
Precision is  96.08150470219435  %
Total time is 33.8550 seconds
